# Intermediate Julia for scientific computing

This workshop is designed to introduce two fundamental concepts in Julia: **types** and **metaprogramming**.

In order to cover various key uses of types in Julia, we have chosen to frame the discussion around a concrete topic in scientific computing, namely **root-finding**. 
The goal is *not* to learn algorithms for root finding *per se*, but rather to have a (pseudo-)real context in which to explore various concepts centered around types and how they arise naturally in real applications of Julia, in particular applications of **multiple dispatch**, which is one of the core choices in Julia that differentiate it from other common languages.

We will implement a couple of root-finding algorithms just to have something to work with. These will just be toy implementations that are far away from the best implementations. 

Instead we should use one of the high-quality packages that are available in Julia for this purpose; the large number of them shows the importance of root finding. I am aware of the following (in alphabetical order):

- Single root of a nonlinear function:
    - [`NLsolve.jl`](https://github.com/JuliaNLSolvers/NLsolve.jl)
    - [`Roots.jl`](https://github.com/JuliaMath/Roots.jl)

- All roots of a polynomial:
    - [`HomotopyContinuation.jl`](https://www.juliahomotopycontinuation.org)
    - [`PolynomialRoots.jl`](https://github.com/giordano/PolynomialRoots.jl)
    - [`Polynomials.jl`](https://github.com/JuliaMath/Polynomials.jl)
    
- All roots of a nonlinear function:
    - [`ApproxFun.jl`](https://github.com/JuliaApproximation/ApproxFun.jl)
    - [`IntervalRootFinding.jl`](https://github.com/JuliaIntervals/IntervalRootFinding.jl)
    - [`MDBM.jl`](https://github.com/bachrathyd/MDBM.jl)
    - [`Roots.jl`](https://github.com/JuliaMath/Roots.jl)

Each of these uses different techniques, with different advantages and disadvantages.

The challenge exercise for the workshop is: develop a package which integrates all of these disparate packages into a coherent whole!

### Logistics of the workshop

The workshop is based around a series of exercises to be done during the workshop. We will pause to work on the exercises and then I will discuss possible solutions during the workshop.

These techniques are useful for both users and developers; indeed, in Julia the distinction between users and developers is not useful, since it's much easier than in other languages to join the two categories together.

### Outline

We will start by quickly reviewing roots of functions and quickly reviewing one of the standard algorithms, **Newton's algorithm**. We will restrict to finding roots of 1D functions for simplicity.

Newton's algorithm requires the calculation of derivatives, for which several choices of algorithm are available. We will see how to encode the choice of algorithm using dispatch.

Then we will define types which will contain all information about a root-finding problem.

## Roots

Given a function $f: \mathbb{R} \to \mathbb{R}$ (i.e. that accepts a single real number as argument and returns another real number), recall that a **root** or **zero** of the function is a number $x^*$ such that

$$ f(x^*) = 0, $$

i.e. it is a solution of the equation $f(x) = 0$.

In general it is impossible to solve this equation exactly for $x^*$, so we use iterative numerical algorithms instead.

#### Example

Recall that the function $f$ given by $f(x) := x^2 - 2$ has exactly two roots, at $x^*_1 = +\sqrt{2}$ and $x^*_2 = -\sqrt{2}$. Note that it is impossible to represent these values exactly using floating-point arithmetic.

## Newton algorithm

The Newton algorithm for (possibly) finding a root of a nonlinear function $f(x)$ in 1D is the following iteration:

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)},$$

where $f'$ is the derivative of $f$. We start from an initial guess $x_0$ that can be almost anything (except points for which $f'(x_0) = 0$).

#### Exercise 1

1. Implement the Newton algorithm for a fixed number $n$ of steps in a function `newton`, starting from a given starting point $x_0$.  

    Hint: Which information does the function require?


2. Does your function work with other number types, such as `BigFloat`? What do you need in order to run it with those types? Use it to calculate $\sqrt{2}$. How many decimal places are correct with the standard precision of `BigFloat`?

#### Solution 1

1. The `newton` function needs the function `f` whose root we wish to find, its derivative `df`, an initial condition `x0`, and a number of iterations `n`:

In [1]:
function newton(f, df, x0, n=10)   # n=10 specifies a default value
    x = x0  # initialise
    
    for i in 1:n
        x_new = x - f(x) / df(x)
        
        x = x_new  # update for next step
    end
    
    return x
end

newton (generic function with 2 methods)

Define a function `g` whose root we wish to find, together with its derivative `dg` specified "by hand":

In [2]:
g(x) = x^2 - 2
dg(x) = 2x

dg (generic function with 1 method)

Run `newton` with these functions and a starting point `y0`:

In [3]:
y0 = 3.0

newton(g, dg, y0)  # we did not need to specify a number of iterations; the default value is used

1.414213562373095

Note that it is very easy to pass functions as arguments to other functions, by specifying the name of the function. This is also efficient.

In [4]:
@code_warntype newton(g, dg, 0.1, 10)

Body::Float64
1 ── %1  = (Base.sle_int)(1, n)::Bool
│    %2  = (Base.ifelse)(%1, n, 0)::Int64
│    %3  = (Base.slt_int)(%2, 1)::Bool
└───       goto #3 if not %3
2 ──       goto #4
3 ──       goto #4
4 ┄─ %7  = φ (#2 => true, #3 => false)::Bool
│    %8  = φ (#3 => 1)::Int64
│    %9  = (Base.not_int)(%7)::Bool
└───       goto #10 if not %9
5 ┄─ %11 = φ (#4 => _4, #9 => %17)::Float64
│    %12 = φ (#4 => %8, #9 => %23)::Int64
│    %13 = (Base.mul_float)(%11, %11)::Float64
│    %14 = (Base.sub_float)(%13, 2.0)::Float64
│    %15 = (Base.mul_float)(2.0, %11)::Float64
│    %16 = (Base.div_float)(%14, %15)::Float64
│    %17 = (Base.sub_float)(%11, %16)::Float64
│    %18 = (%12 === %2)::Bool
└───       goto #7 if not %18
6 ──       goto #8
7 ── %21 = (Base.add_int)(%12, 1)::Int64
└───       goto #8
8 ┄─ %23 = φ (#7 => %21)::Int64
│    %24 = φ (#6 => true, #7 => false)::Bool
│    %25 = (Base.not_int)(%24)::Bool
└───       goto #10 if not %25
9 ──       goto #5
10 ┄ %28 = φ (#8 => %17, #4 => _4):

Note that the functions `f` and `df` were *inlined*: there is no longer a call to the functions, but rather the code for those functions is inserted into the code for the Newton method. Whether this inlining is performed or not is governed by a heuristic based on the code complexity.

Compare with the analytical solution:

In [5]:
sqrt(2)

1.4142135623730951

2. Run the function starting from a `BigFloat` initial condition:

In [6]:
y0 = big(2.0)

2.0

In [7]:
newton(g, dg, y0, 5)

1.414213562373095048801689623502530243614981925776197428498289498623195824228933

In [8]:
result = newton(g, dg, y0, 5)

1.414213562373095048801689623502530243614981925776197428498289498623195824228933

How far is it from the true square root?

In [9]:
result - sqrt(y0)

8.992928321650453100503992493553216097606324633457668310222718673825519970228542e-25

In [10]:
for n in 1:10
    result = newton(g, dg, y0, n)
    println(abs(result - sqrt(y0)))
end

0.08578643762690495119831127579030192143032812462305192682332026200926752153789802
0.00245310429357161786497794245696858809699479128971859348998692867593418820455893
2.123901414755119879903241282313587190869721091142509594771813189090165348662204e-06
1.594861824606854680436831546887746738795971408225281209321893373282554461870123e-12
8.992928321650453100503992493553216097606324633457668310222718673825519970228542e-25
2.859283843333951225327771682478558595433666338812917695204101993287842530821706e-49
0.0
0.0
0.0
0.0


## Calculating derivatives

The Newton algorithm requires us to specify the derivative of a function. If $f$ is a complicated function, we certainly don't want to do that by hand.

One standard solution is to use a *finite-difference approximation*:

$$f'(a) \simeq \frac{f(a + h) - f(a - h)}{2h}.$$

#### Exercise 2

In [11]:
We found the exact

LoadError: syntax: extra token "found" after end of expression

1. Implement a function `finite_difference` with a default value $h = 0.001$.


2. Use an anonymous function to make a method of `finite_difference` that calculates the *function* $f'$.


3. Implement a version of `newton` that does not take the derivative as argument and uses `finite_difference` to calculate the derivative. This version of `newton` should **re-use** the previous version by defining the function `fp` and calling that version.

### Solution 2

1. We copy the mathematical definition. We need the function `f`, the value `a` and the step size `h`, which has a default value:

In [12]:
function finite_difference(f, a, h=0.001)
    return ( f(a + h) - f(a - h) ) / (2h)
end

finite_difference (generic function with 2 methods)

We specify the function and the exact derivatie:

In [13]:
g(x) = x^3 - 2
dg(x) = 3x^2

dg (generic function with 1 method)

In [14]:
a = 3.0

finite_difference(g, a)

27.000000999995777

In [15]:
dg(a)

27.0

How good is the approximation?

In [16]:
dg(a) - finite_difference(g, a)

-9.999957768513923e-7

In [17]:
dg(a) - finite_difference(g, a, 0.1)

-0.01000000000002288

2. Now we want to make a new method of `finite_difference` that takes just the function `f` and returns its derivative. Note that its derivative is *itself a function*, so this will be a function that accepts a function and returns another function:

In [18]:
finite_difference(f) = x -> finite_difference(f, x)

finite_difference (generic function with 3 methods)

Mathematically we can write this as $x \mapsto f\prime(x)$, which we read as "the function that maps any value $x$ to the result of $f'(x)$".

Let's try it out:

In [19]:
dg2 = finite_difference(x->x^2)
dg2(3.0)

5.999999999999339

2. We want to write a "version" (*method*) of the function `newton`, in which we no longer need to specify the derivative by hand, but rather use a finite difference.

Since we already have a version of the `newton` function written which contains the main code for the method, we should try to *re-use* it. (This is the "Don't Repeat Yourself", or DRY, principle.)

In [20]:
function newton(f, x0, n=10)

    # calculate the derivative *function*
    df = finite_difference(f)
    
    # pass it to the previously-defined method of `newton`:
    result = newton(f, df, x0, n)   
    
    return result
end

newton (generic function with 3 methods)

In [21]:
newton(g, 3.0)

1.2599210498948732

In [22]:
cbrt(2)

1.2599210498948732

`newton` is a **generic function**:

In [23]:
@which newton(g, 3.0)

newton(f, x0) in Main at In[20]:4

In [24]:
methods(newton)

# 3 methods for generic function "newton":
[1] newton(f, x0) in Main at In[20]:4
[2] newton(f, x0, n) in Main at In[20]:4
[3] newton(f, df, x0, n) in Main at In[1]:2

Note that default arguments have the effect of creating additional methods.

Things already seem to be getting a bit complicated, since we are building up layers of functions on top of each other. Is this a good idea in terms of performance?

In [25]:
@code_warntype newton(g, finite_difference(g), 3.0, 10)

Body::Float64
1 ── %1  = (Base.sle_int)(1, n)::Bool
│    %2  = (Base.ifelse)(%1, n, 0)::Int64
│    %3  = (Base.slt_int)(%2, 1)::Bool
└───       goto #3 if not %3
2 ──       goto #4
3 ──       goto #4
4 ┄─ %7  = φ (#2 => true, #3 => false)::Bool
│    %8  = φ (#3 => 1)::Int64
│    %9  = (Base.not_int)(%7)::Bool
└───       goto #10 if not %9
5 ┄─ %11 = φ (#4 => _4, #9 => %27)::Float64
│    %12 = φ (#4 => %8, #9 => %33)::Int64
│    %13 = (Base.mul_float)(%11, %11)::Float64
│    %14 = (Base.mul_float)(%13, %11)::Float64
│    %15 = (Base.sub_float)(%14, 2.0)::Float64
│    %16 = (Base.add_float)(%11, 0.001)::Float64
│    %17 = (Base.mul_float)(%16, %16)::Float64
│    %18 = (Base.mul_float)(%17, %16)::Float64
│    %19 = (Base.sub_float)(%18, 2.0)::Float64
│    %20 = (Base.sub_float)(%11, 0.001)::Float64
│    %21 = (Base.mul_float)(%20, %20)::Float64
│    %22 = (Base.mul_float)(%21, %20)::Float64
│    %23 = (Base.sub_float)(%22, 2.0)::Float64
│    %24 = (Base.sub_float)(%19, %23)::Float64
│    

Again everything has got inlined and will be performant!

However, note that a specialised version of `newton` will be compiled for each function that you pass in as we have written it.

In [26]:
newton(x->2x, 3.0)

0.0

### Algorithmic differentiation

An alternative way to calculate derivatives is by using [**algorithmic differentiation**](https://en.wikipedia.org/wiki/Automatic_differentiation) (also called **automatic differentiation** or **computational differentiation**). This gives exact results (up to rounding error).


We will implement this algorithm in the next notebook, but for now let's just use the implementation in the excellent [`ForwardDiff.jl` package](https://github.com/JuliaDiff/ForwardDiff.jl).  A Julia **package** contains Julia code, written as functions, that provides additional functionality that you may use in your code by calling those functions.


#### Exercise 3

1. Install `ForwardDiff.jl` if necessary.


2. Import it.


3. Define a function `forwarddiff` that uses the `ForwardDiff.derivative` function to calculate a derivative.

#### Solution 3

We install the package using the excellent built-in package manager in Julia. If the package is a registered package then Julia automatically knows where to find it and downloads it into a standard location in your local system.  Installing a package is only necessary once in a given installation of Julia.

In [27]:
# using Pkg

# Pkg.add("ForwardDiff")

Alternative:

In [28]:
# ]add ForwardDiff

In order to use the package we must load it. There are two different ways to load a package: `using`, which makes available the functions that the package author has chosen to export by pulling them into your global namespace, and `import`, which does not do so.

In the case of `ForwardDiff`, the functions provided in the package are in any case not exported:

In [29]:
using ForwardDiff   # load the package

We may choose to define our own function to access the relevant function of the package for simplicity:

In [30]:
forwarddiff(f, x) = ForwardDiff.derivative(f, x)

forwarddiff (generic function with 1 method)

Note that the syntax `ForwardDiff.derivative` refers to the function called `derivative` that comes from the `ForwardDiff` package.

In [31]:
ForwardDiff.derivative

derivative (generic function with 4 methods)

In [32]:
forwarddiff(g, a)

27.0

### Choosing between algorithms

We now have two different algorithms available to calculate derivatives. This kind of situation is common in scientific computing; for example, the [`DifferentialEquations.jl`](http://docs.juliadiffeq.org/latest/) ecosystem has some 300 algorithms for solving differential equations. One of the techniques we need to learn is how to specify which algorithm to use.

One possible solution is just by specifying the *function* to use as an argument to another function:

#### Exercise 4

1. Make a version of the Newton algorithm that takes an argument which is the method (algorithm) to use to calculate the derivative, given as a function. 
The new method should have the signature `newton(f, df, x0, n, derivative)`.

#### Solution 4

In [33]:
methods(newton)

# 3 methods for generic function "newton":
[1] newton(f, x0) in Main at In[20]:4
[2] newton(f, x0, n) in Main at In[20]:4
[3] newton(f, df, x0, n) in Main at In[1]:2

In [34]:
function newton(f, df, x0, n, derivative)
    df = x -> derivative(f, x)
    
    return newton(f, df, x0, n)
end

newton (generic function with 4 methods)

In [35]:
newton(g, dg, a, 10, finite_difference)

1.2599210498948732

In [36]:
newton(g, dg, a, 10, forwarddiff)

1.2599210498948732

In [37]:
methods(newton)

# 4 methods for generic function "newton":
[1] newton(f, x0) in Main at In[20]:4
[2] newton(f, x0, n) in Main at In[20]:4
[3] newton(f, df, x0, n) in Main at In[1]:2
[4] newton(f, df, x0, n, derivative) in Main at In[34]:2

We see that the `newton` function API (interface) is getting clumsy: it is difficult to remember which order and what each of the arguments represents. In future notebooks we will see how to improve this.